# Table of Contents
* [Learning Objectives:](#Learning-Objectives:)
* [Plotting with Models](#Plotting-with-Models)
	* [Setting up Data](#Setting-up-Data)
	* [Building the basic plot](#Building-the-basic-plot)
	* [Adding Axes and Grids](#Adding-Axes-and-Grids)
	* [Adding Tools](#Adding-Tools)


# Learning Objectives:

After completion of this module, learners should be able to:

* build complex plots from low-level methods

The raw data sets used in this noteook are stored in the `bokehData` folder in this repository along with importable Python scripts to read the CSV or JSON files into Pandas DataFrames.

Other topics not covered here
* [Widgets](http://bokeh.pydata.org/en/latest/docs/user_guide/interaction.html#adding-widgets)    
* [Javascript Callbacks](http://bokeh.pydata.org/en/latest/docs/user_guide/interaction.html#defining-callbacks)
    * [Callback example](http://nbviewer.ipython.org/github/bokeh/bokeh-notebooks/blob/master/tutorial/A1%20-%20Building%20gapminder.ipynb)
* [Bokeh Sever](http://bokeh.pydata.org/en/latest/docs/user_guide/server.html)
    * [Sliders](http://bokeh.pydata.org/en/latest/docs/server_gallery/sliders_server.html)
    * [Selections](http://bokeh.pydata.org/en/latest/docs/server_gallery/stocks_server.html)

In [ ]:
from bokeh.io import output_notebook, show
output_notebook()

# Plotting with Models

In this example we are going to demonstrate how to build a plot of Usain Bolt vs. 116 years of Olympic sprinters using the most most low-level interface called Models.

As we'll see in this example very little defaults are defined in the Model interface unlike the Figure and Chart interfaces above.

In [ ]:
from data.bokeh.sprint import data as sprint
sprint.head(10)

Next we import some of the Bokeh models that need to be assembled to make a plot. At a minimum, we need to start with Plot, the glyphs (Circle and Text) we want to display, as well as ColumnDataSource to hold the data and range obejcts to set the plot bounds.

In [ ]:
from bokeh.models.glyphs import Circle, Text
from bokeh.models import ColumnDataSource, Range1d, DataRange1d, Plot

## Setting up Data

We want to be able to plot several aspects that are not currently represented in the data. Finally, the DataFrame is converted to a ColumnDataSource. Note that the further we dive into the lower levels of Bokeh the more data manipulation needs to be done by hand.

* Convert the time it took each runner to complete the race to distance behind Usain Bolt
* Set the speed in 100 meters/s
* Set fill_color and line_color data for each element
* Convert abbreviations to full country names
* Select a few names to display

In [ ]:
abbrev_to_country = {
    "USA": "United States",
    "GBR": "Britain",
    "JAM": "Jamaica",
    "CAN": "Canada",
    "TRI": "Trinidad and Tobago",
    "AUS": "Australia",
    "GER": "Germany",
    "CUB": "Cuba",
    "NAM": "Namibia",
    "URS": "Soviet Union",
    "BAR": "Barbados",
    "BUL": "Bulgaria",
    "HUN": "Hungary",
    "NED": "Netherlands",
    "NZL": "New Zealand",
    "PAN": "Panama",
    "POR": "Portugal",
    "RSA": "South Africa",
    "EUA": "United Team of Germany",
}

gold_fill   = "#efcf6d"
gold_line   = "#c8a850"
silver_fill = "#cccccc"
silver_line = "#b0b0b1"
bronze_fill = "#c59e8a"
bronze_line = "#98715d"

fill_color = { "gold": gold_fill, "silver": silver_fill, "bronze": bronze_fill }
line_color = { "gold": gold_line, "silver": silver_line, "bronze": bronze_line }

def selected_name(name, medal, year):
    return name if medal == "gold" and year in [1988, 1968, 1936, 1896] else None

t0 = sprint.Time[0]

sprint["Abbrev"]       = sprint.Country
sprint["Country"]      = sprint.Abbrev.map(lambda abbr: abbrev_to_country[abbr])
sprint["Medal"]        = sprint.Medal.map(lambda medal: medal.lower())
sprint["Speed"]        = 100.0/sprint.Time
sprint["MetersBack"]   = 100.0*(1.0 - t0/sprint.Time)
sprint["MedalFill"]    = sprint.Medal.map(lambda medal: fill_color[medal])
sprint["MedalLine"]    = sprint.Medal.map(lambda medal: line_color[medal])
sprint["SelectedName"] = sprint[["Name", "Medal", "Year"]].apply(tuple, axis=1).map(lambda args: selected_name(*args))

source = ColumnDataSource(sprint)

## Building the basic plot

To build a Plot object ranges have to be provided in the initialization. In order to `show` the plot glyphs must be added.

The Text glyph is used to render information like the few chosen athletes in the ColumnSourceData. Investigate the `sprint` DataFrame to see what is in the `SelectedName` category. You'll notice that the Text glyph coordinates are in data space.

In [ ]:
#prepare a dictionary of plot options to be passed as keyword arguments
plot_options = dict(plot_width=800, plot_height=480, toolbar_location=None, 
                    outline_line_color=None, title = "Usain Bolt vs. 116 years of Olympic sprinters")

In [ ]:
# The Circle and Text models behave like the cirle and text 
#plotting glyphs except that ColumnDataSource object is 
#given when the glyph is added to the plot
radius = dict(value=5, units="screen")
medal_glyph = Circle(x="MetersBack", y="Year", radius=radius, fill_color="MedalFill", 
                     line_color="MedalLine", fill_alpha=0.5)

athlete_glyph = Text(x="MetersBack", y="Year", x_offset=10, text="SelectedName",
    text_align="left", text_baseline="middle", text_font_size="9pt")

no_olympics_glyph = Text(x=7.5, y=1942, text=["No Olympics in 1940 or 1944"],
    text_align="center", text_baseline="middle",
    text_font_size="9pt", text_font_style="italic", text_color="silver")

In [ ]:
# Range1d is a fixed axis range independent of the input data
# DataRange1d is a dynamic axis range that grows with the plotted data.
xdr = Range1d(start=sprint.MetersBack.max()+2, end=0)  # +2 is for padding
ydr = DataRange1d(range_padding=0.05)  

plot = Plot(x_range=xdr, y_range=ydr, **plot_options)
plot.add_glyph(source, medal_glyph)
plot.add_glyph(source, athlete_glyph)
plot.add_glyph(no_olympics_glyph)

In [ ]:
show(plot)

## Adding Axes and Grids

Notice that `x_range` and `y_range` objects were prepared in the plot object but not axes. Ranges are required so that plot can render data in the canvas, but axes are just for style.

In [ ]:
from bokeh.models import Grid, LinearAxis, SingleIntervalTicker

In [ ]:
xdr = Range1d(start=sprint.MetersBack.max()+2, end=0)  # +2 is for padding
ydr = DataRange1d(range_padding=0.05)  

plot = Plot(x_range=xdr, y_range=ydr, **plot_options)
plot.add_glyph(source, medal_glyph)
plot.add_glyph(source, athlete_glyph)
plot.add_glyph(no_olympics_glyph)


#Axes require tickers whose purpose is to set the intervals
#Axes are just for style
xticker = SingleIntervalTicker(interval=5, num_minor_ticks=0)
xaxis = LinearAxis(ticker=xticker, axis_line_color=None, major_tick_line_color=None,
                   axis_label="Meters behind 2012 Bolt", axis_label_text_font_size="10pt", 
                   axis_label_text_font_style="bold")
plot.add_layout(xaxis, "below")

#The ticker is shared with the grid
#Grids are just for style
xgrid = Grid(dimension=0, ticker=xaxis.ticker, grid_line_dash="dashed")
plot.add_layout(xgrid)

yticker = SingleIntervalTicker(interval=12, num_minor_ticks=0)
yaxis = LinearAxis(ticker=yticker, major_tick_in=-5, major_tick_out=10)
plot.add_layout(yaxis, "right")

In [ ]:
show(plot)

## Adding Tools

The final step is to add a HoverTool to help display even more inforamtion than we already have.

In [ ]:
from bokeh.models import HoverTool

In [ ]:
#The tooltips argument allows arbitrary HTML formatting
#The @keyword are columns in the source data
tooltips = """
<div>
    <span style="font-size: 15px;">@Name</span>&nbsp;
    <span style="font-size: 10px; color: #666;">(@Abbrev)</span>
</div>
<div>
    <span style="font-size: 17px; font-weight: bold;">@Time{0.00}</span>&nbsp;
    <span style="font-size: 10px; color: #666;">@Year</span>
</div>
<div style="font-size: 11px; color: #666;">@{MetersBack}{0.00} meters behind</div>
"""

In [ ]:
xdr = Range1d(start=sprint.MetersBack.max()+2, end=0)  # +2 is for padding
ydr = DataRange1d(range_padding=0.05)  

plot = Plot(x_range=xdr, y_range=ydr, **plot_options)
medal = plot.add_glyph(source, medal_glyph)  # we need this renderer to configure the hover tool
plot.add_glyph(source, athlete_glyph)
plot.add_glyph(no_olympics_glyph)

xticker = SingleIntervalTicker(interval=5, num_minor_ticks=0)
xaxis = LinearAxis(ticker=xticker, axis_line_color=None, major_tick_line_color=None,
                   axis_label="Meters behind 2012 Bolt", axis_label_text_font_size="10pt", 
                   axis_label_text_font_style="bold")
plot.add_layout(xaxis, "below")

xgrid = Grid(dimension=0, ticker=xaxis.ticker, grid_line_dash="dashed")
plot.add_layout(xgrid)

yticker = SingleIntervalTicker(interval=12, num_minor_ticks=0)
yaxis = LinearAxis(ticker=yticker, major_tick_in=-5, major_tick_out=10)
plot.add_layout(yaxis, "right")

In [ ]:
# The HoverTool is rendered only for the the medal_glyph and none of the two text glyphs
hover = HoverTool(tooltips=tooltips, renderers=[medal])
plot.add_tools(hover)

In [ ]:
show(plot)